## Установка зависимостей

In [ ]:
!pip install selenium
!pip install webdriver_manager
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

## Запуск браузера

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

## Scrapping

In [ ]:
import time
LOGIN = ''
PASSWORD = ''

driver.set_window_size(2000, 1000)

driver.get('https://familio.org')

time.sleep(5)

wait = WebDriverWait(driver, 20)

login_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@data-testid="login-button"]')))
login_button.click()

username_input = driver.find_element(By.NAME, 'login')
password_input = driver.find_element(By.NAME, 'password')

username_input.send_keys(LOGIN)
password_input.send_keys(PASSWORD)

submit_button = driver.find_element(By.CSS_SELECTOR, '.loginForm-submitButton')
submit_button.click()

time.sleep(25)


driver.set_window_size(600, 600)

In [ ]:
import time
import csv

start_page = 10101
end_page   = 25200
pages_per_file = 100


pages_to_scrap = list(range(start_page, end_page+1))
page_groups = [pages_to_scrap[i:i+pages_per_file] for i in range(0, len(pages_to_scrap), pages_per_file)]

for page_group in page_groups:
    data = []
    for page_num in page_group:

        driver.get('https://familio.org/knowledge-base/catalogs/duma1906?page-records=' + str(page_num))


        print('Processing page ' + str(page_num))

        rows = driver.find_elements(By.CSS_SELECTOR, '.catalogTable-tableRow')

        # Skip the first row
        for row in rows[1:]:
            link = row.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')

            driver.execute_script(f'window.open("{link}", "_blank");')

            driver.switch_to.window(driver.window_handles[-1])



            data_elements = driver.find_elements(By.CSS_SELECTOR, '.catalogPerson-data dl dt, .catalogPerson-data dl dd')
            data.append({data_elements[i].text: data_elements[i+1].text for i in range(0, len(data_elements), 2)})


            driver.close()

            driver.switch_to.window(driver.window_handles[0])



    csv_file = "drive/MyDrive/scrapper-data_old/{}-{}.csv".format(page_group[0], page_group[-1])

    with open(csv_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=data[0].keys())
        writer.writeheader()
        writer.writerows(data)


driver.quit()